In [23]:
import codecs
import multiprocessing
import os
import pprint
import re
import nltk
import gensim.models.word2vec as w2v
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [24]:
nltk.download("punkt")             # For tokenization
nltk.download("stopwords")         #For removing unnecessary words

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\0x6f736f646f\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\0x6f736f646f\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
with open("../Data/Combined.txt") as f:
    data = f.read()

In [26]:
tokenized_sent = nltk.tokenize.word_tokenize(language="english", text=data)

In [27]:
def sentence_to_word(corpus):
    clean = re.sub("[^a-zA-Z]"," ", corpus)
    words = clean.split()
    words = [word.lower() for word in words]
    return words

In [28]:
#Converting every sentence to tokens
sentences = []
for tokenized_s in tokenized_sent:
    if len(tokenized_s) > 0:
        sentences.append(sentence_to_word(tokenized_s))

In [29]:
token_count = sum([len(sentence) for sentence in sentences])
print("The corpus contains {0:,} tokens".format(token_count))

The corpus contains 614,242 tokens


In [35]:
num_features = 500

min_word_count = 3

#number of parallel workers
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
downsampling = 1e-3

seed = 42

In [36]:
model = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

C:\Users\0x6f736f646f\AppData\Local\Programs\Python\Python37\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


In [37]:
    The model type (continuous bag of words  skipgram typically is favored for large corpus
    Sampling method ( hierarchical or negative ) again latter typically preferred for large corpus
    Subsampling - determines how often frequently occurring words like “the” are trained. Could be anywhere from 1e-3 to 1e-5. Values lower than 1e-5 tends to influence vector quality
    Window size - smaller window size gives results that are more syntactic in nature. Larger window ( size > 5 ) gives results that are more semantic - this takes more training time given larger window size.

In [37]:
model.build_vocab(sentences)

In [38]:
print("len of vocab is %d" %(len(model.wv.vocab)))

len of vocab is 11888


In [39]:
model.train(sentences,total_examples=model.corpus_count, epochs=model.iter)

C:\Users\0x6f736f646f\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(2395073, 3071210)

In [40]:
if not os.path.exists("../model"):
    os.makedirs("../model")

In [41]:
model.save(os.path.join("../model", "model.w2v"))